In [1]:
# one-time: PEFT + bitsandbytes + trl
!pip install -q peft==0.10.0 trl==0.8.6 bitsandbytes==0.45.5 transformers==4.51.3 accelerate==1.7.0 datasets==2.20.0 sqlalchemy==2.0.30 sqlite-utils==3.38 evaluate==0.4.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88

In [2]:
!pip list | grep -E 'peft|trl|triton|bitsandbytes|transformers|accelerate|datasets|sqlalchemy|sqlite-utils|moz-sql-parser|evaluate|torch|torchvision'

accelerate                            1.7.0
bitsandbytes                          0.45.5
datasets                              2.20.0
evaluate                              0.4.3
fastrlock                             0.8.3
peft                                  0.10.0
sentence-transformers                 4.1.0
sqlite-utils                          3.38
tensorflow-datasets                   4.9.8
torch                                 2.6.0+cu124
torchao                               0.10.0
torchaudio                            2.6.0+cu124
torchdata                             0.11.0
torchsummary                          1.5.1
torchtune                             0.6.1
torchvision                           0.21.0+cu124
transformers                          4.51.3
triton                                3.2.0
trl                                   0.8.6
vega-datasets                         0.9.0


In [3]:
%%bash
python - <<'PY'
import random, numpy as np, torch, os
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
print("Seeds fixed to", SEED)
PY


Seeds fixed to 42


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from huggingface_hub import login

login()

In [5]:
from datasets import load_dataset

spider_train   = load_dataset("spider")["train"]
spider_val = load_dataset("spider")["validation"]

print(spider_train)
print(spider_val)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 7000
})
Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 1034
})


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "deepseek-ai/deepseek-coder-6.7b-base"  # use preferred model
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model      = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)
model.eval()

tokenizer_config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-06)
    (rotary_e

In [ ]:
!pip install -q kagglehub            # tiny, pure-python package
import kagglehub, os, json, zipfile
from pathlib import Path

# Either:
#   kagglehub.login()                              # will prompt
#   – or –
os.environ["KAGGLE_USERNAME"] = ""        # <- set once per session
os.environ["KAGGLE_KEY"]     = ""

In [8]:
import kagglehub, zipfile
from pathlib import Path

root = Path( kagglehub.dataset_download(
           "jeromeblanchet/yale-universitys-spider-10-nlp-dataset") )

# 1️⃣  Look for a zip first (old mirror versions)
zip_files = list(root.rglob("spider.zip"))
if zip_files:
    print("Found spider.zip – extracting…")
    with zipfile.ZipFile(zip_files[0], "r") as zf:
        zf.extractall("spider")          # creates ./spider/database/…
    SPIDER_DIR = Path("spider")

# 2️⃣  Otherwise assume the folder is already there (current mirror)
else:
    print("No spider.zip – using unpacked folder")
    # the mirror root itself is usually “…/spider/”, but we search just in case
    try:
        SPIDER_DIR = next(root.rglob("spider/database")).parent
    except StopIteration as e:
        raise FileNotFoundError(
            "Could not locate Spider data inside the mirror download") from e

DB_ROOT = SPIDER_DIR / "database"
print("Spider databases live at:", DB_ROOT.resolve())


No spider.zip – using unpacked folder
Spider databases live at: /kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/database


In [9]:
DB_ROOT = Path("/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/database")

In [10]:
#   reflect every .sqlite file to create a plain-text schema description


import sqlalchemy as sa

_schema_cache: dict[str, str] = {}            # {db_id: "…schema string…"}

def get_schema_string(db_id: str) -> str:
    """Return a compact textual schema for the given Spider database."""
    if db_id in _schema_cache:
        return _schema_cache[db_id]

    db_file = DB_ROOT / db_id / f"{db_id}.sqlite"
    engine  = sa.create_engine(f"sqlite:///{db_file}")
    insp    = sa.inspect(engine)

    parts   = []
    for tbl in sorted(insp.get_table_names()):
        cols = [c["name"] for c in insp.get_columns(tbl)]
        parts.append(f"{tbl}({', '.join(cols)})")

    schema_str = ", ".join(parts)
    _schema_cache[db_id] = schema_str
    return schema_str

In [11]:
from datasets import load_dataset
import random

# Use any split you like – here we sample 5 from Spider-train
NUM_SHOTS      = 5
spider_train   = load_dataset("spider")["train"]

random.seed(42
            )
# Turn each row into a normal dict and attach the schema string
DEMO_SET = [
    {
        **ex,                                  # copy all original fields
        "schema_str": get_schema_string(ex["db_id"])
    }
    for ex in spider_train.shuffle(seed=42).select(range(NUM_SHOTS))
]


In [12]:
from datasets import load_dataset, DatasetDict

spider_train = load_dataset("spider")["train"]
half_size    = len(spider_train) // 5                      # 20 %
train_subset = spider_train.shuffle(seed=42).select(range(half_size))

def make_prompt(example):
    schema = get_schema_string(example["db_id"])
    return (
        "### Instruction:\n"
        "You are an expert SQL developer. Given a database schema and a natural-language\n"
        "question, write ONE syntactically correct SQL query that answers the question.\n"
        "Return **only** the SQL; do not repeat the schema or add explanations.\n\n"

        f"### Database Schema:\n{schema}\n\n"

        f"### Question:\n{example['question']}\n\n"

        "### SQL:\n"
        f"{example['query'].strip()}"
    )

train_subset = train_subset.map(lambda ex: {"text": make_prompt(ex)}) \
                           .remove_columns(set(train_subset.column_names) - {"text"})

# sanity:
print(train_subset[0]["text"][:400], "...")
print("Fine-tune set size:", len(train_subset))


Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

<ipython-input-10-a06efc3c7c6c>:19: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['11']; using no arguments.
  cols = [c["name"] for c in insp.get_columns(tbl)]
<ipython-input-10-a06efc3c7c6c>:19: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.BIGINT'> with reflected arguments ['20']; using no arguments.
  cols = [c["name"] for c in insp.get_columns(tbl)]


### Instruction:
You are an expert SQL developer. Given a database schema and a natural-language
question, write ONE syntactically correct SQL query that answers the question.
Return **only** the SQL; do not repeat the schema or add explanations.

### Database Schema:
player(Player_ID, Player, Team, Age, Position, School_ID), school(School_ID, School, Location, Enrollment, Founded, Denomination, B ...
Fine-tune set size: 1400


In [13]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit               = True,
    bnb_4bit_use_double_quant  = True,
    bnb_4bit_quant_type        = "nf4",
    bnb_4bit_compute_dtype     = torch.bfloat16,   # bf16 saves VRAM vs fp16
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_cfg,
    device_map="auto",
)
tokenizer.pad_token_id = tokenizer.eos_token_id


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
from transformers import BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, TaskType, get_peft_model
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model, TaskType

lora_cfg = LoraConfig(
    r                = 64,
    lora_alpha       = 128,
    target_modules   = ["q_proj","k_proj","v_proj","o_proj"],
    lora_dropout     = 0.05,
    bias             = "none",
    task_type        = TaskType.CAUSAL_LM,
)

peft_model = get_peft_model(base_model, lora_cfg)

# ↓ this halves activation memory at the cost of ~15 % speed
peft_model.gradient_checkpointing_enable()
peft_model.enable_input_require_grads()

In [15]:
from transformers import TrainingArguments
from trl import SFTTrainer           # same as before

args = TrainingArguments(
    output_dir                  = "/content/deepseek_coder_spider_qlora",
    per_device_train_batch_size = 2,          # ↓ was 4
    gradient_accumulation_steps = 8,          # effective batch still 16
    num_train_epochs            = 5,          # your new request
    learning_rate               = 2e-4,
    fp16                        = False,      # bfloat16 already set
    bf16                        = True,
    logging_steps               = 50,
    save_strategy               = "epoch",
    save_total_limit            = 2,
    seed                        = 42,
)

trainer = SFTTrainer(
    model              = peft_model,
    tokenizer          = tokenizer,
    train_dataset      = train_subset,
    dataset_text_field = "text",
    args               = args,
    max_seq_length     = 2048,
)

trainer.train()
trainer.save_model(args.output_dir)     # LoRA adapters + config only


Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:323: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a d

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: johneze (john-eze) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
50,0.580700
100,0.250500
150,0.168600
200,0.137100
250,0.120600
300,0.109600
350,0.101800
400,0.092100


In [17]:
print("GPU Mem Peak:", torch.cuda.max_memory_allocated() / 1e9, "GB")

GPU Mem Peak: 21.307646976 GB


In [16]:
train_subset

Dataset({
    features: ['text'],
    num_rows: 1400
})

In [18]:
import torch, gc
from peft import PeftModel


#  reload quantised base model (4-bit)
base_model = AutoModelForCausalLM.from_pretrained(
    lora_cfg.base_model_name_or_path, # Changed from peft_cfg to lora_cfg
    quantization_config=bnb_cfg,
    device_map={"": 0},          # ← keep all layers on GPU 0
    torch_dtype=torch.bfloat16,
)

#  attach LoRA adapters
# Need to define output_dir before using it
output_dir = "/content/deepseek_coder_spider_qlora" # Define output_dir

ft_model = PeftModel.from_pretrained(base_model, output_dir)  # no device_map needed
ft_model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit(

In [19]:
print("GPU Mem Peak:", torch.cuda.max_memory_allocated() / 1e9, "GB")

GPU Mem Peak: 24.926410752 GB


# Introducing zero-shot prompting to the Qlora Adapters

In [20]:
# ──────────────────────────────────────────────────────────────────────────
# 0 ▪ global seed (reproducible shuffles, torch, numpy, random, etc.)
# ──────────────────────────────────────────────────────────────────────────
import random, numpy as np, torch, os
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

# ──────────────────────────────────────────────────────────────────────────
# 1 ▪ zero-shot prompt builder (no demos)
# ──────────────────────────────────────────────────────────────────────────
def build_prompt_zero(nl_question: str, target_schema: str) -> str:
    return (
        "### Instruction:\n"
        "You are an expert SQL developer. Given the database schema and the "
        "question, return ONE valid SQL statement — output ONLY the SQL.\n\n"
        f"### Database Schema:\n{target_schema}\n\n"
        f"### Question:\n{nl_question}\n\n"
        "### SQL:\n"
    )

# ──────────────────────────────────────────────────────────────────────────
# 2 ▪ random-5-shot builder
# ──────────────────────────────────────────────────────────────────────────
NUM_SHOTS = 5
spider_train = load_dataset("spider")["train"]
DEMO_SET = [
    {**ex, "schema_str": get_schema_string(ex["db_id"])}
    for ex in spider_train.shuffle(seed=SEED).select(range(NUM_SHOTS))
]

def build_prompt_random5(nl_question: str, target_schema: str) -> str:
    parts = ["### Instruction:\nReturn ONE SQL query only, based on the examples provided.\n"]

    for i, ex in enumerate(DEMO_SET, 1):
        parts += [
            f"### Example {i} Schema:\n{ex['schema_str']}",
            f"### Example {i} Question:\n{ex['question']}",
            f"### Example {i} SQL:\n{ex['query'].strip()}",
        ]
    parts += [
        f"### Database Schema:\n{target_schema}",
        f"### Question:\n{nl_question}",
        "### SQL:\n",
    ]
    return "\n".join(parts)

# ──────────────────────────────────────────────────────────────────────────
# 3 ▪ retrieved-5-shot builder (SBERT nearest neighbours)
# ──────────────────────────────────────────────────────────────────────────
from sentence_transformers import SentenceTransformer, util
embedder   = SentenceTransformer("all-MiniLM-L6-v2")
train_emb  = embedder.encode(spider_train["question"], convert_to_tensor=True)

def get_k_shots(nl_question: str, k: int = 5):
    q_emb  = embedder.encode(nl_question, convert_to_tensor=True)
    hits   = util.semantic_search(q_emb, train_emb, top_k=k)[0]
    demos  = []
    for h in hits:
        ex = spider_train[int(h["corpus_id"])]
        demos.append({
            "question"   : ex["question"],
            "query"      : ex["query"],
            "schema_str" : get_schema_string(ex["db_id"]),
        })
    return demos

def build_prompt_retrieved5(nl_question: str, target_schema: str) -> str:
    demos = get_k_shots(nl_question, 5)
    parts = ["### Instruction:\nReturn ONE SQL query only, based on the examples provided.\n"]

    for i, ex in enumerate(demos, 1):
        parts += [
            f"### Example {i} Schema:\n{ex['schema_str']}",
            f"### Example {i} Question:\n{ex['question']}",
            f"### Example {i} SQL:\n{ex['query'].strip()}",
            "### End\n",
        ]
    parts += [
        f"### Database Schema:\n{target_schema}",
        f"### Question:\n{nl_question}",
        "### SQL:\n",
    ]
    return "\n".join(parts)

# ──────────────────────────────────────────────────────────────────────────
# 4 ▪ shared generate_sql that accepts a *builder* argument
# ──────────────────────────────────────────────────────────────────────────
import re

def make_generator(builder_fn):
    def _gen(nl_question: str, schema: str) -> str:
        prompt = builder_fn(nl_question, schema)
        inputs = tokenizer(prompt, return_tensors="pt").to(ft_model.device)
        input_len = inputs["input_ids"].shape[1]

        ft_model.config.pad_token_id = tokenizer.eos_token_id
        out = ft_model.generate(
            **inputs,
            max_new_tokens=128,
            num_beams=3,
            early_stopping=True,
            do_sample=False,
        )
        gen_text = tokenizer.decode(out[0, input_len:], skip_special_tokens=True)
        sql = re.split(r"(###|\n\s*\n|```)", gen_text, 1)[0]
        sql = sql.split(";")[0].replace("\n", " ").strip()
        return sql.split(";", 1)[0].strip()
    return _gen

# choose the prompt mode here ─────────────────────────────────────────────
PROMPT_MODE = 0          # 0 = zero-shot, 1 = random-5, 2 = retrieved-5
builder = [build_prompt_zero,
           build_prompt_random5,
           build_prompt_retrieved5][PROMPT_MODE]
generate_sql = make_generator(builder)

# ──────────────────────────────────────────────────────────────────────────
# 5 ▪ run the usual 20 % dev evaluation (exact same loop you used before)
# ──────────────────────────────────────────────────────────────────────────
# … sample_val creation …
# … em_scores, cm_scores, ex_scores, times …
# loop exactly as before – now uses the new generate_sql


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
print("GPU Mem Peak:", torch.cuda.max_memory_allocated() / 1e9, "GB")

GPU Mem Peak: 24.926410752 GB


In [20]:
# choose the prompt mode here ─────────────────────────────────────────────
PROMPT_MODE = 2          # 0 = zero-shot, 1 = random-5, 2 = retrieved-5
builder = [build_prompt_zero,
           build_prompt_random5,
           build_prompt_retrieved5][PROMPT_MODE]
generate_sql = make_generator(builder)


sample = spider_val[19]
schema = get_schema_string(sample["db_id"])
print(sample["question"])
print(sample['query'])
print(generate_sql(sample["question"], schema))


Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


What is the name and capacity for the stadium with the highest average attendance?
SELECT name ,  capacity FROM stadium ORDER BY average DESC LIMIT 1
SELECT name ,  capacity FROM stadium ORDER BY average_attendance DESC LIMIT 1


# Defining Evaluation Metrics

In [22]:
import re
_ws   = re.compile(r"\s+")
_comm = re.compile(r"\s*,\s*")          # blank(s) ↔ comma ↔ blank(s)

def exact_match(pred: str, gold: str) -> bool:
    def norm(s: str) -> str:
        s = s.strip().lower()
        s = _ws.sub(" ", s)             # collapse runs of whitespace
        s = _comm.sub(", ", s)          # canonical “comma␣”
        return s
    return norm(pred) == norm(gold)




from sqlglot import parse_one, expressions

def flatten_ast(node):
    """
    Recursively collect all node‐type names and literal values as lowercase strings.
    """
    out = set()

    def walk(n):
        # record the AST node type
        out.add(type(n).__name__.lower())

        # record any literal (e.g. identifiers, strings, numbers)
        if hasattr(n, "this") and isinstance(n.this, (str, int, float)):
            out.add(str(n.this).lower())

        # recurse into child expressions
        for arg in n.args.values():
            if isinstance(arg, list):
                for child in arg:
                    if isinstance(child, expressions.Expression):
                        walk(child)
            elif isinstance(arg, expressions.Expression):
                walk(arg)

    walk(node)
    return out

def component_match(pred_sql, gold_sql):
    try:
        pred_ast = parse_one(pred_sql)
        gold_ast = parse_one(gold_sql)
    except Exception:
        return 0.0

    pred_set = flatten_ast(pred_ast)
    gold_set = flatten_ast(gold_ast)
    if not gold_set:
        return 0.0
    return len(pred_set & gold_set) / len(gold_set)




import sqlite3, pandas as pd, numpy as np
from tqdm import tqdm

def run_query(sql: str, db_path: Path):
    """Return query result as a sorted list of tuples (order-independent)."""
    try:
        with sqlite3.connect(db_path) as conn:
            df = pd.read_sql_query(sql, conn)
        # sort rows + cols for order-invariant comparison
        return tuple(map(tuple, df.sort_index(axis=1).sort_values(list(df.columns)).to_numpy()))
    except Exception as e:
        # any failure counts as wrong
        return f"ERROR-{e}"

def execution_accuracy(dataset):
    """Compute Spider-style Execution Accuracy on a HF split (e.g. validation)."""
    correct = 0
    for ex in tqdm(dataset, desc="Evaluating"):
        db_id   = ex["db_id"]
        schema  = get_schema_string(db_id)
        pred_sql= generate_sql(ex["question"], schema)

        db_file = DB_ROOT / db_id / f"{db_id}.sqlite"
        gold    = run_query(ex["query"],     db_file)
        pred    = run_query(pred_sql,        db_file)

        if gold == pred:
            correct += 1

    return correct / len(dataset)

# Quick Sanity Check before evaluation

In [22]:
# Sanity Check
import time, torch, numpy as np
from tqdm import tqdm
import warnings
from transformers import logging

random.seed(42)

logging.set_verbosity_error()
warnings.filterwarnings('ignore')


# choose the prompt mode here ─────────────────────────────────────────────
PROMPT_MODE = 2          # 0 = zero-shot, 1 = random-5, 2 = retrieved-5
builder = [build_prompt_zero,
           build_prompt_random5,
           build_prompt_retrieved5][PROMPT_MODE]
generate_sql = make_generator(builder)


sample = spider_val[19]
schema = get_schema_string(sample["db_id"])
print(sample["question"])
print(sample['query'])
print(generate_sql(sample["question"], schema))


predicted_sql = generate_sql(sample["question"], schema)
golden_sql = sample["query"]
exact_match_score = exact_match(predicted_sql, golden_sql)
print(f"Exact Match: {exact_match_score}")
component_match_score = component_match(predicted_sql, golden_sql)
print(f"Component Match: {component_match_score}")
sample_dataset = spider_val.select(range(5))
exec_acc = execution_accuracy(sample_dataset)
print(f"Execution Accuracy: {exec_acc}")

What is the name and capacity for the stadium with the highest average attendance?
SELECT name ,  capacity FROM stadium ORDER BY average DESC LIMIT 1
SELECT name ,  capacity FROM stadium ORDER BY average_attendance DESC LIMIT 1
Exact Match: False
Component Match: 0.9285714285714286


Evaluating: 100%|██████████| 5/5 [01:21<00:00, 16.28s/it]

Execution Accuracy: 1.0


# Eval for Zero-shot prompt

In [23]:
# Evaluation on a fixed 20% Spider dev sample
import time, torch, numpy as np
from tqdm import tqdm
import warnings
from transformers import logging

random.seed(42)

logging.set_verbosity_error()
warnings.filterwarnings('ignore')


# choose the prompt mode here ─────────────────────────────────────────────
PROMPT_MODE = 0          # 0 = zero-shot, 1 = random-5, 2 = retrieved-5
builder = [build_prompt_zero,
           build_prompt_random5,
           build_prompt_retrieved5][PROMPT_MODE]
generate_sql = make_generator(builder)


# 1) Size of a 20 % slice (Spider dev has 1 034 samples)
num_samples = int(0.20 * len(spider_val))      # → 206

sample_val  = spider_val.shuffle(seed=42).select(range(num_samples))


# 2) containers
em_scores, cm_scores, ex_scores, times = [], [], [], []

# 3) start fresh CUDA-peak tracking
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()

# 4) main loop
for ex in tqdm(sample_val, desc="Evaluating"):
    db_id   = ex["db_id"]
    schema  = get_schema_string(db_id)

    t0      = time.perf_counter()
    pred_sql= generate_sql(ex["question"], schema)
    times.append(time.perf_counter() - t0)

    gold_sql= ex["query"]

    # exact + component match
    em_scores.append( float(exact_match(pred_sql, gold_sql)) )
    cm_scores.append( component_match(pred_sql, gold_sql) )

    # execution accuracy
    db_path = DB_ROOT / db_id / f"{db_id}.sqlite"
    gold_res= run_query(gold_sql,  db_path)
    pred_res= run_query(pred_sql, db_path)
    ex_scores.append( int(gold_res == pred_res) )

# 5) aggregate & report
print("\n\nExact Match        :", np.mean(em_scores))
print("Component Match    :", np.mean(cm_scores))
print("Execution Accuracy :", np.mean(ex_scores))
print("Avg. Latency  (s)  :", np.mean(times))
print("95% Latency  (s)   :", np.percentile(times, 95))
if torch.cuda.is_available():
    print("GPU Mem Peak       :", torch.cuda.max_memory_allocated() / 1e9, "GB")
else:
    print("GPU Mem Peak       : N/A (CPU)")


Evaluating: 100%|██████████| 206/206 [45:45<00:00, 13.33s/it]



Exact Match        : 0.27184466019417475
Component Match    : 0.9175878606117407
Execution Accuracy : 0.6067961165048543
Avg. Latency  (s)  : 13.21514432043689
95% Latency  (s)   : 14.75402082024982
GPU Mem Peak       : 23.448722944 GB


In [24]:
print("GPU Mem Peak:", torch.cuda.max_memory_allocated() / 1e9, "GB")

GPU Mem Peak: 23.448722944 GB


# Introducing 5-shot prompt (Random Sample and MiniLM) on the Qlora Adapters

# Evaluation for 5-shot (Random) prompt for Qlora Adapters

In [25]:
# Evaluation on a fixed 20% Spider dev sample
import time, torch, numpy as np
from tqdm import tqdm
import warnings
from transformers import logging

random.seed(42)

logging.set_verbosity_error()
warnings.filterwarnings('ignore')


# choose the prompt mode here ─────────────────────────────────────────────
PROMPT_MODE = 1          # 0 = zero-shot, 1 = random-5, 2 = retrieved-5
builder = [build_prompt_zero,
           build_prompt_random5,
           build_prompt_retrieved5][PROMPT_MODE]
generate_sql = make_generator(builder)


# 1) Size of a 20 % slice (Spider dev has 1 034 samples)
num_samples = int(0.20 * len(spider_val))      # → 206

sample_val  = spider_val.shuffle(seed=42).select(range(num_samples))


# 2) containers
em_scores, cm_scores, ex_scores, times = [], [], [], []

# 3) start fresh CUDA-peak tracking
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()

# 4) main loop
for ex in tqdm(sample_val, desc="Evaluating"):
    db_id   = ex["db_id"]
    schema  = get_schema_string(db_id)

    t0      = time.perf_counter()
    pred_sql= generate_sql(ex["question"], schema)
    times.append(time.perf_counter() - t0)

    gold_sql= ex["query"]

    # exact + component match
    em_scores.append( float(exact_match(pred_sql, gold_sql)) )
    cm_scores.append( component_match(pred_sql, gold_sql) )

    # execution accuracy
    db_path = DB_ROOT / db_id / f"{db_id}.sqlite"
    gold_res= run_query(gold_sql,  db_path)
    pred_res= run_query(pred_sql, db_path)
    ex_scores.append( int(gold_res == pred_res) )

# 5) aggregate & report
print("\n\nExact Match        :", np.mean(em_scores))
print("Component Match    :", np.mean(cm_scores))
print("Execution Accuracy :", np.mean(ex_scores))
print("Avg. Latency  (s)  :", np.mean(times))
print("95% Latency  (s)   :", np.percentile(times, 95))
if torch.cuda.is_available():
    print("GPU Mem Peak       :", torch.cuda.max_memory_allocated() / 1e9, "GB")
else:
    print("GPU Mem Peak       : N/A (CPU)")


Evaluating: 100%|██████████| 206/206 [53:12<00:00, 15.50s/it]



Exact Match        : 0.23786407766990292
Component Match    : 0.9369069585190832
Execution Accuracy : 0.587378640776699
Avg. Latency  (s)  : 15.388941398310605
95% Latency  (s)   : 15.647215007249542
GPU Mem Peak       : 24.415657472 GB


# Evaluation for 5-shot (all-MiniLM) prompt for Qlora Adapters

In [26]:
# Evaluation on a fixed 20% Spider dev sample
import time, torch, numpy as np
from tqdm import tqdm
import warnings
from transformers import logging

random.seed(42)

logging.set_verbosity_error()
warnings.filterwarnings('ignore')


# choose the prompt mode here ─────────────────────────────────────────────
PROMPT_MODE = 2          # 0 = zero-shot, 1 = random-5, 2 = retrieved-5
builder = [build_prompt_zero,
           build_prompt_random5,
           build_prompt_retrieved5][PROMPT_MODE]
generate_sql = make_generator(builder)


# 1) Size of a 20 % slice (Spider dev has 1 034 samples)
num_samples = int(0.2 * len(spider_val))      # → 206

sample_val  = spider_val.shuffle(seed=42).select(range(num_samples))


# 2) containers
em_scores, cm_scores, ex_scores, times = [], [], [], []

# 3) start fresh CUDA-peak tracking
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()

# 4) main loop
for ex in tqdm(sample_val, desc="Evaluating"):
    db_id   = ex["db_id"]
    schema  = get_schema_string(db_id)

    t0      = time.perf_counter()
    pred_sql= generate_sql(ex["question"], schema)
    times.append(time.perf_counter() - t0)

    gold_sql= ex["query"]

    # exact + component match
    em_scores.append( float(exact_match(pred_sql, gold_sql)) )
    cm_scores.append( component_match(pred_sql, gold_sql) )

    # execution accuracy
    db_path = DB_ROOT / db_id / f"{db_id}.sqlite"
    gold_res= run_query(gold_sql,  db_path)
    pred_res= run_query(pred_sql, db_path)
    ex_scores.append( int(gold_res == pred_res) )

# 5) aggregate & report
print("\n\nExact Match        :", np.mean(em_scores))
print("Component Match    :", np.mean(cm_scores))
print("Execution Accuracy :", np.mean(ex_scores))
print("Avg. Latency  (s)  :", np.mean(times))
print("95% Latency  (s)   :", np.percentile(times, 95))
if torch.cuda.is_available():
    print("GPU Mem Peak       :", torch.cuda.max_memory_allocated() / 1e9, "GB")
else:
    print("GPU Mem Peak       : N/A (CPU)")


Evaluating: 100%|██████████| 206/206 [55:17<00:00, 16.11s/it]



Exact Match        : 0.2912621359223301
Component Match    : 0.9344112908549725
Execution Accuracy : 0.6116504854368932
Avg. Latency  (s)  : 16.009534365830024
95% Latency  (s)   : 17.268238604001
GPU Mem Peak       : 31.67950592 GB
